In [21]:
import json

file = open('translation2019zh_train.json' , 'r',encoding='utf-8') 

en=[]
ch=[]

for line in file.readlines():
    tmp=json.loads(line)
    en.append(tmp['english'])
    ch.append(tmp['chinese'])

#print(en[5161433])
#print(ch[5161433])
print(en[:10])
print(ch[:10])

file.close()

['For greater sharpness, but with a slight increase in graininess, you can use a 1:1 dilution of this developer.', 'He calls the Green Book, his book of teachings, “the new gospel.', 'And the light breeze moves me to caress her long ear', 'They have the blood of martyrs is the White to flow …', "Finally, the Lakers head to the Motor City to take on a Pistons team that currently owns the Eastern Conference's second best record (1/31). L.", '"The perfect match—my father loves names and Jackie loves money, " sneered Alexander at the wedding. Neither he nor Christina ever got along with their stepmother17.', 'In 2006, Walmart was charged with racism when its recommendation engine paired Planet of the Apes with a documentary about Martin Luther King.', 'The matte as main copper phase in the cleaning. slag was deter- mined by electron probe microscopic analysis.', 'Have you shined your shoes?', 'The Tanning Matrix can be formed by resorcinol and oxazolidine E, and the reactioncharateristics 

In [23]:
# 分別生成中英文字典
en_vocab = set(''.join(en))
id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}

# 分別生成中英文字典
ch_vocab = set(''.join(ch))
id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}

print(en2id)

{'经': 0, '鸽': 1, '督': 2, '坎': 3, '舅': 4, '隔': 5, '榈': 6, '宫': 7, '令': 8, '吟': 9, '骆': 10, '喊': 11, '屡': 12, '乞': 13, '蓄': 14, '研': 15, '巫': 16, '锡': 17, '毅': 18, '腿': 19, '临': 20, '立': 21, '发': 22, '杏': 23, '钢': 24, '渐': 25, '罩': 26, '稀': 27, '桦': 28, '宀': 29, '琉': 30, '涡': 31, '积': 32, '塘': 33, '＂': 34, '口': 35, '黛': 36, '娱': 37, '骑': 38, '汽': 39, '斜': 40, '浏': 41, '坞': 42, '歇': 43, '贷': 44, '参': 45, '爽': 46, '蝌': 47, '面': 48, '氮': 49, '汀': 50, '俾': 51, '犊': 52, '救': 53, '糊': 54, '勘': 55, 'Ⅷ': 56, '礼': 57, '裹': 58, '蹒': 59, '脸': 60, '毁': 61, 'U': 62, '升': 63, '脚': 64, '潮': 65, '伍': 66, '收': 67, '励': 68, '足': 69, '∂': 70, '瞥': 71, '台': 72, '弗': 73, '斩': 74, '撤': 75, '芝': 76, '湿': 77, '闭': 78, '偿': 79, '略': 80, '板': 81, '餐': 82, '拍': 83, '即': 84, '全': 85, '掺': 86, '突': 87, '吩': 88, '狮': 89, '尊': 90, '暇': 91, '冤': 92, '胡': 93, '烘': 94, '挞': 95, '威': 96, '沸': 97, '蝇': 98, '填': 99, '残': 100, '园': 101, '蘸': 102, '风': 103, '▼': 104, '踏': 105, '奥': 106, '聪': 107, '察': 108, '币': 109, '裤': 110,